## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# import sys
# sys.path.append('../../../')

from openbb_terminal import api as openbb
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = openbb.widgets.html_report_stylesheet()
with open("./openbb_terminal/jupyter/reports/OpenBB_reports_logo.png", "rb") as image_file:
    openbb_image_encoded = base64.b64encode(image_file.read())

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "TSLA"
report_name = ""

In [ ]:
ticker_data = openbb.stocks.load(ticker, start=datetime.datetime.now() - datetime.timedelta(days=4*30))
ticker_data = openbb.stocks.process_candle(ticker_data)

author = "Didier Rodrigues Lopes"
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date =  datetime.datetime.now().strftime('%d %B, %Y')
report_time =  datetime.datetime.now().strftime('%H:%M') 
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
overview = openbb.stocks.fa.models.yahoo_finance.get_info(ticker=ticker).transpose()[
    "Long business summary"
][0]
overview

## Data

In [ ]:
(
    df_year_estimates,
    df_quarter_earnings,
    df_quarter_revenues,
) = openbb.stocks.dd.models.business_insider.get_estimates(ticker)

In [ ]:
display_year = sorted(df_year_estimates.columns.tolist())[:3]
df_year_estimates = df_year_estimates[display_year].head(5)
df_year_estimates

In [ ]:
df_quarter_earnings

In [ ]:
df_quarter_revenues

In [ ]:
df_sec_filings = openbb.stocks.dd.models.marketwatch.get_sec_filings(ticker)[
    ["Type", "Category", "Link"]
].head(5)
df_sec_filings

In [ ]:
df_analyst = openbb.stocks.dd.models.finviz.get_analyst_data(ticker)
df_analyst["target_to"] = df_analyst["target_to"].combine_first(df_analyst["target"])
df_analyst = df_analyst[["category", "analyst", "rating", "target_to"]].rename(
    columns={
        "category": "Category",
        "analyst": "Analyst",
        "rating": "Rating",
        "target_to": "Price Target",
    }
)
df_analyst

In [ ]:
fig, ax1 = plt.subplots(figsize=(11, 5), dpi=150)
ax2 = ax1.twinx()
openbb.stocks.dps.spos(
    ticker,
    num=84,
    raw=False,
    export="",
    external_axes=[ax1, ax2],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
net_short_position = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.dps.dpotc(
    ticker,
    external_axes=[ax1, ax2],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
dark_pools = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
openbb.stocks.dps.volexch(
    ticker,
    external_axes=[ax],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
volexch = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
ax3 = ax1.twinx()
openbb.stocks.dps.psi_sg(
    ticker,
    external_axes=[ax1, ax2, ax3],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_vs_short_interest = f.getvalue().decode("utf-8")

In [ ]:
fig, (candles, volume) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.candle(
    s_ticker=ticker,
    df_stock=ticker_data,
    use_matplotlib=True,
    external_axes=[candles, volume],
)
candles.set_xticklabels("")
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_chart = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.dd.pt(
    ticker=ticker,
    start="2022-01-01",
    interval="1440min",
    stock=ticker_data,
    num=10,
    raw=False,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
price_target_chart = f.getvalue().decode("utf-8")

In [ ]:
df = openbb.stocks.dd.models.business_insider.get_price_target_from_analysts(ticker)
avg_ratings_last_30_days = round(np.mean(df[datetime.datetime.now() - datetime.timedelta(days=30):]["Price Target"].values), 2)
last_price = round(ticker_data["Close"][-1], 2)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.dd.rot(
    ticker=ticker,
    num=10,
    raw=False,
    export="",
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ratings_over_time_chart = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 3), dpi=150)
openbb.stocks.ta.rsi(ticker_data["Close"], external_axes=[ax1, ax2])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ta_rsi = f.getvalue().decode("utf-8")

In [ ]:
df = openbb.stocks.ta.models.momentum.rsi(ticker_data["Close"])
rsi_value = round(df.values[-1][0], 2)
rsi_value

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(np.array(range(len(ticker_data["Close"][-30:].index))).reshape(-1,1), ticker_data["Close"][-30:].values)
regression_slope = round(model.coef_[0], 2)

In [ ]:
import pandas as pd
df_insider = pd.DataFrame.from_dict(openbb.stocks.ins.models.finviz.get_last_insider_activity(ticker)).head(10)
df_insider["Val ($)"] = df_insider["Value ($)"].replace({',': ''}, regex=True)
df_insider["Trade"] = df_insider.apply(
        lambda row: (-1*float(row["Val ($)"])) if row["Transaction"] == "Sale" else (float(row["Val ($)"]) if row["Transaction"] == "Buy" else 0),
        axis=1,
    )
last_10_insider_trading = round(sum(df_insider["Trade"])/1_000_000, 2)
df_insider = df_insider.drop(columns=["Val ($)", "Trade"])
df_insider

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.ba.headlines(ticker, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
finbrain_sentiment = f.getvalue().decode("utf-8")

In [ ]:
df_sentiment_finbrain = openbb.stocks.ba.models.finbrain.get_sentiment(ticker)
finbrain_sentiment_val = float(df_sentiment_finbrain.values[-1][0])
finbrain_sentiment_val = 10

In [ ]:
watchlist_count, n_cases, n_bull, n_bear = openbb.stocks.ba.models.stocktwits.get_bullbear(ticker)
stocktwits_sentiment = f"Watchlist count: {watchlist_count}</br>"
if n_cases > 0:
    stocktwits_sentiment += f"\nLast {n_cases} sentiment messages:</br>"
    stocktwits_sentiment += f"Bullish {round(100*n_bull/n_cases, 2)}%</br>"
    stocktwits_sentiment += f"Bearish {round(100*n_bear/n_cases, 2)}%"
else:
    stocktwits_sentiment += "No messages found"

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.ba.snews(ticker, external_axes=[ax1, ax2])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
snews = f.getvalue().decode("utf-8")

In [ ]:
ticker_data_all = openbb.stocks.load(ticker, start=datetime.datetime.now() - datetime.timedelta(days=5*12*21))
ticker_data_all["Returns"] = ticker_data_all["Adj Close"].pct_change()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.qa.bw(ticker, ticker_data_all, "Returns", False, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw_month = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.qa.bw(ticker, ticker_data_all, "Returns", True, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw_year = f.getvalue().decode("utf-8")

In [ ]:
regression_val = round(openbb.stocks.pred.models.regression.get_regression_model(ticker_data_all["Close"], 1, 80, 20, 1)[0][-1], 2)
regression_val

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.pred.regression(ticker, ticker_data_all["Close"], 1, 80, 20, 1, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
regression = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(11, 3), dpi=150)
openbb.stocks.pred.mc(ticker_data["Close"], 30, 100, external_axes=[ax1, ax2])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
mc = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.pred.regression(ticker, ticker_data_all["Close"], 1, 80, 20, 1, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
regression = f.getvalue().decode("utf-8")

## Render the report template to a file

In [ ]:
body = ""
body += f'''
    <div style="display:flex; margin-bottom:1cm;">
        <img src="data:image/png;base64,{openbb_image_encoded.decode()}" alt="OpenBB" style="width:144px;height:90px;">
        <div style="margin-left:2em">
            <p><b>Analyst:</b> {author}</p>
            <p><b>Date   :</b> {report_date}</p>
            <p><b>Time   :</b> {report_time} {report_timezone}</p>
            <br/>
            <p><b>INVESTMENT RESEARCH REPORT:</b> {ticker}</p>
        </div>
    </div>'''

body += '''
    <div class="tab">
      <button class="tablinks" onclick="menu(event, 'SUMMARY')">SUMMARY</button>
      <button class="tablinks" onclick="menu(event, 'Overview')">Overview</button>
      <button class="tablinks" onclick="menu(event, 'Analysts Opinions')">Analysts Opinions</button>
      <button class="tablinks" onclick="menu(event, 'Fundamental Analysis')">Fundamental Analysis</button>
      <button class="tablinks" onclick="menu(event, 'Dark Pool and Shorts')">Dark Pool and Shorts</button>
      <button class="tablinks" onclick="menu(event, 'Technical Analysis')">Technical Analysis</button>
      </br>
      <button class="tablinks" onclick="menu(event, 'Insider Trading')">Insider Trading</button>
      <button class="tablinks" onclick="menu(event, 'Behavioural Analysis')">Behavioural Analysis</button>
      <button class="tablinks" onclick="menu(event, 'Quantitative Analysis')">Quantitative Analysis</button>
      <button class="tablinks" onclick="menu(event, 'Prediction Techniques')">Prediction Techniques</button>      
    </div>'''

body += '<div id="SUMMARY" class="tabcontent">'
body += '<h3>KPIs:</h3>'
if avg_ratings_last_30_days < last_price:
    body += f'<p style="color:red">&#10060; Last closing price ({last_price}) > Average price ratings last 30 days ({avg_ratings_last_30_days})</p>'
else:
    body += f'<p style="color:green">&#x2705; Average price ratings last 30 days ({avg_ratings_last_30_days}) > Last closing price ({last_price})</p>'

if regression_slope > 0:
    body += f'<p style="color:green">&#x2705; Regression on the last 30 market days ({regression_slope} dollars/market day) is positive</p>'
else:
    body += f'<p style="color:red">&#10060; Regression on the last 30 market days ({regression_slope} dollars/market day) is negative</p>'
    
if rsi_value < 30:
    body += f'<p style="color:green">&#x2705; RSI level is OVERSOLD. {rsi_value} < 30</p>'
elif rsi_value > 70:
    body += f'<p style="color:red">&#10060; RSI level is OVERBOUGHT. {rsi_value} > 70</p>'
else:
    body += f'<p style="color:orange">&#128993; RSI level is NORMAL. 30 < {rsi_value} < 70</p>'
    
if last_10_insider_trading > 0:
    body += f'<p style="color:green">&#x2705; Last 10 insider trading. {last_10_insider_trading} M was bought in value.</p>'
else:
    body += f'<p style="color:red">&#10060; Last 10 insider trading. {last_10_insider_trading} M was sold in value</p>'
    
if finbrain_sentiment_val > .1:
    body += f'<p style="color:green">&#x2705; Last FinBrain sentiment is BULLISH: {finbrain_sentiment_val} </p>'
elif finbrain_sentiment_val < -.1:
    body += f'<p style="color:red">&#10060; Last FinBrain sentiment is BEARISH: {finbrain_sentiment_val} </p>'
else:
    body += f'<p style="color:orange">&#128993; Last FinBrain sentiment is NEUTRAL: {finbrain_sentiment_val} </p>'
    
if (regression_val - last_price) > 0:
    body += f'<p style="color:green">&#x2705; Regression for next 20th market price ({regression_val}) > Last closing price ({last_price})</p>'
else:
    body += f'<p style="color:red">&#10060; Regression for next 20th market price ({regression_val}) < Last closing price ({last_price})</p>'
    
body += '<h3>Opinion:</h3>'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true";></p>'
body += '</div>'

body += '<div id="Overview" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([price_chart])
body += openbb.widgets.row([openbb.widgets.h(3, "Description") + openbb.widgets.p(overview)])
body += '</div>'

body += '<div id="Analysts Opinions" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([price_target_chart])
body += openbb.widgets.row([ratings_over_time_chart])
body += openbb.widgets.row([df_analyst.to_html()])
body += '</div>'

body += '<div id="Fundamental Analysis" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([openbb.widgets.h(3, "Estimates") + df_year_estimates.head().to_html()])
body += openbb.widgets.row([openbb.widgets.h(3, "Earnings") + df_quarter_earnings.head().to_html()])
body += openbb.widgets.row([openbb.widgets.h(3, "Revenues") + df_quarter_revenues.head().to_html()])
body += openbb.widgets.row([openbb.widgets.h(3, "SEC filings") + df_sec_filings.head().to_html()])
body += '</div>'

body += '<div id="Dark Pool and Shorts" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([net_short_position])
body += openbb.widgets.row([price_vs_short_interest])
body += openbb.widgets.row([dark_pools])
body += openbb.widgets.row([volexch])
body += '</div>'

body += '<div id="Technical Analysis" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([ta_rsi])
body += '</div>'

body += '<div id="Insider Trading" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([openbb.widgets.h(3, "Last Activity") + df_insider.head(10).to_html(col_space='75px')])
body += '</div>'


body += '<div id="Behavioural Analysis" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([finbrain_sentiment])
body += openbb.widgets.row([snews])
body += openbb.widgets.row([openbb.widgets.h(3, "Stocktwits") + stocktwits_sentiment])
body += '</div>'

body += '<div id="Quantitative Analysis" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([bw_month])
body += openbb.widgets.row([bw_year])
body += '</div>'

body += '<div id="Prediction Techniques" class="tabcontent">'
body += '</br> <p style="border:3px; border-style:solid; border-color:#000000; padding: 1em; width: 1050px;" contentEditable="true">No comment.</p>'
body += openbb.widgets.row([regression])
body += openbb.widgets.row([mc])
body += '</div>'


body += '''
    <script>
    function menu(evt, menu_name) {
      var i, tabcontent, tablinks;
      tabcontent = document.getElementsByClassName("tabcontent");
      for (i = 0; i < tabcontent.length; i++) {
        tabcontent[i].style.display = "none";
      }
      tablinks = document.getElementsByClassName("tablinks");
      for (i = 0; i < tablinks.length; i++) {
        tablinks[i].className = tablinks[i].className.replace(" active", "");
        tablinks[i].style.backgroundColor = "white";
        tablinks[i].style.color = "black";
      }
      document.getElementById(menu_name).style.display = "block";
      
      evt.currentTarget.className += " active";
      evt.currentTarget.style.backgroundColor = "black";
      evt.currentTarget.style.color = "white";
    }

    window.onload=function(){
        menu(event, 'SUMMARY');
    };
    </script>'''

report = openbb.widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)
# to save the results
with open(report_name + ".html", "w") as fh:
    fh.write(report)